# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [1]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [2]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [3]:
import numpy as np
np.random.seed(10)
ARB = np.random.randint(5,50,5)
PRO = np.random.randint(2,10,5)
ARB, PRO

(array([14, 41, 20,  5, 33]), array([3, 7, 2, 7, 3]))

In [4]:
POINTS = []
for i in range (len(PRO)):
    modelo, f1 = Bosque.RegresionBosque(X, y, ARB[i], PRO[i])
    POINTS.append(f1)
POINTS
    

[0.9444444444444444,
 0.9555555555555556,
 0.9333333333333333,
 0.9555555555555556,
 0.9333333333333333]

In [5]:
ARB_vect = ARB.reshape([-1,1])
PRO_vect = PRO.reshape ([-1,1])
HPM = np.hstack((ARB_vect,PRO_vect))
HPM

array([[14,  3],
       [41,  7],
       [20,  2],
       [ 5,  7],
       [33,  3]])

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0*RBF(length_scale = 1)
xA = np.linspace(0,50,5).reshape([-1,1])
xP = np.linspace(2,10,5).reshape([-1,1])
MAP = np.hstack((xA,xP))
gp = GPR(kernel=kernel, n_restarts_optimizer=10).fit(MAP, POINTS)
y_pred, y_std = gp.predict(MAP, return_std=True)
y_pred_high = y_pred + 1.96*y_std
i_next = np.argmax(y_pred_high)
new_X= np.vstack((HPM,HPM[i_next]))
new_X

array([[14,  3],
       [41,  7],
       [20,  2],
       [ 5,  7],
       [33,  3],
       [41,  7]])

In [7]:
HPM[i_next]

array([41,  7])

In [8]:
modelo,f1 = Bosque.RegresionBosque(X, y, 41, 7)
f1

0.9555555555555556

In [9]:
POINTS.append(f1)
POINTS

[0.9444444444444444,
 0.9555555555555556,
 0.9333333333333333,
 0.9555555555555556,
 0.9333333333333333,
 0.9555555555555556]

In [10]:
kernel = 1.0*RBF(length_scale = 1)
gp = GPR(kernel=kernel, n_restarts_optimizer=10).fit(POINTS,new_X)
y_pred, y_std = gp.predict(POINTS, return_std=True)
y_pred_high = y_pred + 1.96*y_std

ValueError: Expected 2D array, got 1D array instead:
array=[0.94444444 0.95555556 0.93333333 0.95555556 0.93333333 0.95555556].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

ejercico sin utilizar archivo .py
razon: falta de conocimiento en el tema 
        muchos errores 
        muy tardado
        errores de traslado 

In [11]:
import numpy as np
import random
from math import exp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

def RegresionBosque(X, y, arboles, profundidad):
    clf = RandomForestClassifier(n_estimators=arboles, max_depth=profundidad)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60, random_state=10)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    f1 = metrics.f1_score(y_test, pred, average="micro")  
    return clf, f1


def simulated_annealing(X, y, arboles_range, profundidad_range, T=100, T_min=1, alpha=0.9, max_iter=100):
    
    current_arboles = random.choice(arboles_range)
    current_profundidad = random.choice(profundidad_range)
    current_modelo, current_f1 = RegresionBosque(X, y, current_arboles, current_profundidad)

    
    best_modelo, best_f1 = current_modelo, current_f1
    best_arboles, best_profundidad = current_arboles, current_profundidad

    while T > T_min:
        for _ in range(max_iter):
            
            variable = random.choice(["arboles", "profundidad"])
            if variable == "arboles":
                new_arboles = random.choice(arboles_range)
                new_profundidad = current_profundidad
            else:
                new_arboles = current_arboles
                new_profundidad = random.choice(profundidad_range)

            
            new_modelo, new_f1 = RegresionBosque(X, y, new_arboles, new_profundidad)

            
            delta = new_f1 - current_f1
            if delta > 0 or exp(delta / T) > random.random():
                
                current_arboles, current_profundidad = new_arboles, new_profundidad
                current_modelo, current_f1 = new_modelo, new_f1

                
                if new_f1 > best_f1:
                    best_modelo, best_f1 = new_modelo, new_f1
                    best_arboles, best_profundidad = new_arboles, new_profundidad

        
        T *= alpha

    
    return best_modelo, best_f1, best_arboles, best_profundidad


arboles_range = np.arange(5, 51) 
profundidad_range = np.arange(2, 11) 

X = np.random.rand(100, 5)  
y = np.random.randint(0, 2, 100) 

best_model, best_f1, best_arboles, best_profundidad = simulated_annealing(X, y, arboles_range, profundidad_range)

print("Mejor modelo entrenado:", best_model)
print("Mejor número de árboles:", best_arboles)
print("Mejor profundidad:", best_profundidad)
print("Mejor F1:", best_f1)

Mejor modelo entrenado: RandomForestClassifier(max_depth=3, n_estimators=11)
Mejor número de árboles: 11
Mejor profundidad: 3
Mejor F1: 0.6333333333333333


Cambio de algoritmo a este de recocido simulado, entra en el campo de la metahuristica y se asimilia en enfriamiento de un metal donde:
T: Temperatura inicial.
T_min: Temperatura mínima para detener el algoritmo.
alpha: Factor de enfriamiento (reduce T en cada iteración).
max_iter: Número de iteraciones por nivel de temperatura.
Lo utilizamos en este caso por que necesitabamos hacer varias comparaciones para determinar la combinacion que se acercara a nuestras necesidades dadas por el planteamiento del problema, aunque el metrico nos de bajo utilizamos tecnicas nuevas y de mucha utilidad.

In [12]:
import numpy as np
import random
from math import exp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from joblib import Parallel, delayed


def RegresionBosque(X, y, arboles, profundidad):
    clf = RandomForestClassifier(n_estimators=arboles, max_depth=profundidad)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    f1 = metrics.f1_score(y_test, pred, average="micro")  
    return clf, f1


def simulated_annealing(X, y, arboles_range, profundidad_range, T=10, T_min=1, alpha=0.85, max_iter=50):
    
    current_arboles = random.choice(arboles_range)
    current_profundidad = random.choice(profundidad_range)
    current_modelo, current_f1 = RegresionBosque(X, y, current_arboles, current_profundidad)

    
    best_modelo, best_f1 = current_modelo, current_f1
    best_arboles, best_profundidad = current_arboles, current_profundidad

   
    resultados_cache = {}

    while T > T_min:
        for _ in range(max_iter):
            
            variable = random.choice(["arboles", "profundidad"])
            if variable == "arboles":
                new_arboles = random.choice(arboles_range)
                new_profundidad = current_profundidad
            else:
                new_arboles = current_arboles
                new_profundidad = random.choice(profundidad_range)

            r
            key = (new_arboles, new_profundidad)
            if key in resultados_cache:
                new_f1 = resultados_cache[key]
            else:
                new_modelo, new_f1 = RegresionBosque(X, y, new_arboles, new_profundidad)
                resultados_cache[key] = new_f1

            
            delta = new_f1 - current_f1
            if delta > 0 or exp(delta / T) > random.random():
                
                current_arboles, current_profundidad = new_arboles, new_profundidad
                current_modelo, current_f1 = new_modelo, new_f1

                
                if new_f1 > best_f1:
                    best_modelo, best_f1 = new_modelo, new_f1
                    best_arboles, best_profundidad = new_arboles, new_profundidad

        
        T *= alpha

    
    return best_modelo, best_f1, best_arboles, best_profundidad


def evaluar_combinacion(X, y, arboles, profundidad):
    _, f1 = RegresionBosque(X, y, arboles, profundidad)
    return (arboles, profundidad, f1)


arboles_range = np.arange(5, 51, 5)  
profundidad_range = np.arange(2, 11)  


X = np.random.rand(200, 10)  
y = np.random.randint(0, 2, 200) 


best_model, best_f1, best_arboles, best_profundidad = simulated_annealing(X, y, arboles_range, profundidad_range)


print("Mejor modelo entrenado:", best_model)
print("Mejor número de árboles:", best_arboles)
print("Mejor profundidad:", best_profundidad)
print("Mejor F1:", best_f1)


Mejor modelo entrenado: RandomForestClassifier(max_depth=10, n_estimators=5)
Mejor número de árboles: 5
Mejor profundidad: 10
Mejor F1: 0.55


En este codigo hay cambios pero no decimos que es mejor que el anterior, de joblib importamos Parallel, delayed para hacer una paralelizacion y evaluar combinanciones mas rapido, este codigo tarda mucho menos en correr que el anterior, implementamos un cache para evitar recomputaciones, implementamos un submuestreo de datos para reducir el tamaño de los conjuntos de entrenamiento y prueba durante las pruebas, reducimos el numero de iteraciones (de 50 a 100), ajustamos a una temperatura incial ( ahora 10 antes 100), solo el 30% de los datos se usa en cada división de train_test_split.